<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Answers_7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

# Exercise 7.2 Answers

In this exercise we build a word language model using a recurrent network. We test it on a Cloze task in which we have to choose which word fits best within a given sentence.


(a) Setup libraries.

In [0]:
import requests
import numpy as np

%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN, LSTM, GRU, Bidirectional, Dropout, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import get_file


---
(b) Load the text we will use for training. This is a collection of stories from the Gutenberg archive. Run the code and add comments.

In [0]:
# download the training text
url = "https://www.phon.ucl.ac.uk/courses/pals0039/data/cloze-corpus.txt"
response = requests.get(url)
# convert the text to a single lower case string
raw_text = response.text.lower().replace('\n',' ')
print("Corpus has",len(raw_text),"characters")
print(raw_text[:250])

---
(c) Tokenize the text. Run the code and add comments.

In [0]:
# set the vocabulary limit to be 10,000 words
max_words=10000

# use the Keras Tokenizer
tokenizer = Tokenizer(num_words=max_words,oov_token="UNK")
tokenizer.fit_on_texts([raw_text])
word_index=tokenizer.word_index
print("Found",len(word_index),"different words.")

In [0]:
# print the 10 most common and 10 least common words
print(list(word_index.items())[:10])
print(list(word_index.items())[-10:])

In [0]:
# convert the text to a list of word indexes, replacing rare words with UNK
raw_seq=tokenizer.texts_to_sequences([raw_text])[0]
# print the start of the sample
print(raw_seq[:50])
# report largest code
print("# words",len(raw_seq),"Max index",max(raw_seq))
# calculate the out-of-vocabulary rate
num_oov=sum(1 for w in raw_seq if w==1)
print("%OOV",100*num_oov/len(raw_seq))

---
(d) Prepare the text sample for training. Run the code and add comments.

In [0]:
# divide the text into sequences of fixed length
seq_len=100
nseq=len(raw_seq)//seq_len
# chunk the text into sequences
seq=np.reshape(raw_seq[:nseq*seq_len],(nseq,seq_len))
# shift the text back one word
raw_seq_shift=np.roll(raw_seq,-1) 
# and chunk into sequences to act as targets
seq_shift=np.reshape(raw_seq_shift[:nseq*seq_len],(nseq,seq_len))
# randomise the order of the sequences
p = np.random.permutation(nseq)
seq=seq[p]
seq_shift=seq_shift[p];

# divide into train and validation sets
nval=nseq//10
Xval=seq[:nval,:]
yval=seq_shift[:nval,:]
Xtrain=seq[nval:,:]
ytrain=seq_shift[nval:,:]

print(Xtrain.shape,ytrain.shape)
print(Xval.shape,yval.shape)

print(Xtrain[0,:10],ytrain[0,:10])


---
(e) Build a recurrent model. Run the code and add comments.

In [0]:
import tensorflow as tf
# define a function for Keras to report perplexity during training
def perplexity(y_true, y_pred):
    cross_entropy = tf.losses.sparse_categorical_crossentropy(y_true, y_pred)
    perplexity = tf.exp(tf.reduce_mean(cross_entropy))
    return perplexity

# set up basic sizes for nextwork
isize=max_words
embed_size=64
osize=max_words

# build the model with an embedding layer and two layers of LSTMs
model = Sequential()
model.add(Embedding(input_dim=isize, output_dim=embed_size,input_length=seq_len))
model.add(LSTM(32,return_sequences=True,activation='tanh'))
model.add(LSTM(32,return_sequences=True,activation='tanh'))
model.add(TimeDistributed(Dense(osize, activation='softmax')));
#
# compile the network
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=[perplexity])
print(model.summary())

---
(f) Train the model. Experiment with different network configurations and training protocol. There is also code here to save and restore trained models since training can take some time.

In [0]:
# train the model
history=model.fit(Xtrain,ytrain, batch_size=64, validation_data=(Xval,yval), epochs=25)


In [0]:
# (optional) save the model to your Google drive account
from google.colab import drive
drive.mount('/content/gdrive')
model_save_name = 'ex72.h5'
path = "/content/gdrive/My Drive/"+model_save_name
model.save(path,)

In [0]:
# (optional) load a trained model from your Google drive account
from google.colab import drive
drive.mount('/content/gdrive')
model_save_name = 'ex72.h5'
path = "/content/gdrive/My Drive/"+model_save_name
model=load_model(path, custom_objects={'perplexity': perplexity})

In [0]:
# (optional) load a trained model from the course website
model_save_name = 'ex72.h5'
url = "https://www.phon.ucl.ac.uk/courses/pals0039/data/"+model_save_name
file = get_file(model_save_name,url,cache_subdir="models")
model=load_model(file, custom_objects={'perplexity': perplexity})


---
(g) Load the data for cloze task to use for testing. Run the code and add comments.

In [0]:
import pandas as pd

# read in the Cloze data set from the course web site.
df=pd.read_csv("https://www.phon.ucl.ac.uk/courses/pals0039/data/cloze-test.csv",keep_default_na=False)

# the basics of the Cloze task is made up of the columns: df.CONTEXT, df.QUERY and df.ALTERNATIVES with the correct answer in df.ANSWER
df.head()

---
(h) Encode the cloze test data using the tokenizer and assemble into sequences. Run the code and add comments.

In [0]:
# concatenate the context and the query and divide up the alternatives
cloze_context=[]
cloze_answer=[]
cloze_alter=[]
for i in range(len(df)):
  # concatenate the CONTEXT and the QUERY to get sufficient text for the LM
  str=df.CONTEXT.iat[i]+" "+df.CONTEXT.iat[i]+" "+df.QUERY.iat[i]
  cloze_context.append(str)
  cloze_answer.append(df.ANSWER.iat[i])
  cloze_alter.append((df.ALTERNATIVES.iat[i]).split('|'))

# convert the strings to integer sequences
cloze_context_seq=tokenizer.texts_to_sequences(cloze_context)
cloze_answer_seq=tokenizer.texts_to_sequences(cloze_answer)
cloze_alter_seq=tokenizer.texts_to_sequences(cloze_alter)

In [0]:
# print some samples of what we have
print(cloze_context[0])
print(cloze_context_seq[0])
print(cloze_answer[:10])
print(cloze_answer_seq[:10])
print(cloze_alter[:10])
print(cloze_alter_seq[:10])


---
(i) Run the model over the all the test sequences and obtain a pdf over the word that completes the query. Run the code and add comments. 

In [0]:
seq_len=100
# chop all context sequences down to seq_len values by taking the last 100 words
cloze_context_lim=np.stack(np.array([ x[-seq_len:] for x in cloze_context_seq]))
print(cloze_context_lim.shape)

In [0]:
# run the model in batches of 100 so as not to overload memory
block_size=50
nblock=cloze_context_lim.shape[0]//block_size
# array to hold the word probabilities
ypred=np.zeros((nblock*block_size,max_words))
for i in range(nblock):
  # get the predicted word probabilities
  testdata=cloze_context_lim[i*block_size:(i+1)*block_size,:]
  pred=model.predict(testdata,batch_size=50)
  # save the probabilities for the last predicted word
  ypred[i*block_size:(i+1)*block_size,:]=pred[:,-1,:]

print(ypred.shape)


---
(j) For each cloze sentence, find the probabilities of each of the alternatives and choose the most probable. Run the code and add comments.

In [0]:
# for each cloze sentence in turn
ntest=ypred.shape[0]
ncorrect=0;
for i in range(ntest):
  # get the number of alternatives
  nprob=len(cloze_alter_seq[i])
  # get the predicted probabilities
  prob=np.zeros(nprob)
  for j in range(nprob):
    prob[j]=ypred[i,cloze_alter_seq[i][j]]
  # choose the most probable among the alternatives
  top_word=cloze_alter_seq[i][np.argmax(prob)]
  # get the actually correct word
  correct_word=cloze_answer_seq[i][0]
  # record whether we got it right
  if (top_word==correct_word):
    ncorrect += 1

# print how well we did
print("Correct: %d/%d (%.1f%%)" % (ncorrect,ntest,100*ncorrect/ntest))
